In [82]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import time

In [98]:
strava_foundation_date = datetime.strptime("2009-01-01", "%Y-%m-%d")

In [56]:
segments = pd.read_csv("../dataset/strava-segments-italy_only-FINAL-GEOCODED.csv")

In [57]:
segments[["s_lat", "s_lng"]] = segments["start_latlng"].apply(lambda x: pd.Series(np.array(eval(x))))

In [58]:
segments[["e_lat", "e_lng"]] = segments["end_latlng"].apply(lambda x: pd.Series(np.array(eval(x))))

In [59]:
segments = segments.drop(columns=["start_latlng", "end_latlng"])

In [60]:
segments.drop("Unnamed: 0", axis=1, inplace=True)

In [61]:
segments.to_csv(path_or_buf="../dataset/segments-italy-PREPROCESSED.csv", index=True)

In [62]:
activities = pd.read_csv("../dataset/strava-activities-italy-full-FINAL.csv")

In [63]:
activities.drop("Unnamed: 0", axis=1, inplace=True)
activities.drop("Unnamed: 0.1", axis=1, inplace=True)

In [64]:
activities = activities.dropna()

In [65]:
activities["segment"] = activities["segment"].astype(int, copy=True)

In [66]:
activities[activities["start_date"].isnull()]["start_date"]

Series([], Name: start_date, dtype: object)

In [67]:
indexes_with_wrong_dates = activities[activities["start_date"].str.match(r"[234][1-9]\d\d-*")].index

In [68]:
activities.drop(indexes_with_wrong_dates, inplace=True)

In [69]:
indexes_with_wrong_dates = activities[activities["start_date"].str.match(r"[4]\d\d\d-*")].index

In [70]:
activities.drop(indexes_with_wrong_dates, inplace=True)

In [71]:
activities["start_date"] = pd.to_datetime(activities["start_date"])

In [72]:
 activities["Year"] = activities["start_date"].dt.year

In [73]:
 activities["Month"] = activities["start_date"].dt.month

In [74]:
 activities["Day"] = activities["start_date"].dt.day

In [75]:
 activities["Dow"] = activities["start_date"].dt.dayofweek

Removing impossible-date activities: future dates as well as dates for activities prior to Strava launch (even though these may be legal, but we expect to be a unrelevant figure)

In [88]:
index_activities_future_dates = activities[activities.start_date > datetime.now()].index

In [89]:
activities.drop(index_activities_future_dates, inplace=True)

In [99]:
index_activities_prior_foundation_date = activities[activities.start_date < strava_foundation_date].index

In [100]:
activities.drop(index_activities_prior_foundation_date, inplace=True)

In [103]:
activities_parsed = pd.merge(activities, segments, left_on="segment", right_on="id").drop(["id", "points"], axis=1)

In [107]:
activities_parsed.to_csv(path_or_buf="../dataset/activities-italy-PREPROCESSED.csv", index=False)